# Introduction

In this assignment you will be given a series of tasks about using the library `power-grid-model` and performing a
**state estimation**. In this assignment we will use the output data of a power flow calculation as input for our sensors
so we can perform a state estimation. The tasks include:

* Think about the number of sensors needed
* Print the required loadflow output
* Initializing only the required sensors and performing a state estimation
* Comparing the state estimation results to the loadflow output
* Adding more sensors to make the calculation more accurate

The network we'll be using for troughout this workshop consists of three nodes, two lines, two loads and a source as shown below:

```
 node_1 ---<line_4>--- node_2 ---<line_5>--- node_3
   |                     |                     |
source_8               load_6                load_7
```

# Preparation

First import everything we need for this workshop:

In [ ]:
from typing import Dict

import numpy as np
import pandas as pd

from IPython.display import display, Markdown

from power_grid_model import (
    PowerGridModel,
    CalculationType,
    CalculationMethod,
    ComponentType,
    DatasetType,
    LoadGenType,
    initialize_array,
    
)

from power_grid_model.validation import (
    assert_valid_input_data,
    assert_valid_batch_data
)

## Generate example data

In order to use realistic data in out workshop, we use the results of a power flow calculation, which gives us voltages, voltage angles, active powers, reactive powers, etc. Below all components of the network are initialized, a model is composed and a power flow calculation is executed.

In [ ]:
# Initialize 3 nodes
node = initialize_array(DatasetType.input, ComponentType.node, 3)
node["id"] = [1, 2, 3]
node["u_rated"] = [10.5e3, 10.5e3, 10.5e3]

# Initialize 2 lines between the 3 nodes
line = initialize_array(DatasetType.input, ComponentType.line, 2)
line["id"] = [4, 5]
line["from_node"] = [1, 2]
line["to_node"] = [2, 3]
line["from_status"] = [1, 1]
line["to_status"] = [1, 1]
line["r1"] = [0.25, 0.25]
line["x1"] = [0.2, 0.2]
line["c1"] = [10e-6, 10e-6]
line["tan1"] = [0.0, 0.0]
line["i_n"] = [1000, 1000]

# Initialize 2 loads, each connected to a different node
sym_load = initialize_array(DatasetType.input, ComponentType.sym_load, 2)
sym_load["id"] = [6, 7]
sym_load["node"] = [2, 3]
sym_load["status"] = [1, 1]
sym_load["type"] = [LoadGenType.const_power, LoadGenType.const_power]
sym_load["p_specified"] = [20e6, 10e6]
sym_load["q_specified"] = [5e6, 2e6]

# Initialize 1 source, connected to a different node than the loads
source = initialize_array(DatasetType.input, ComponentType.source, 1)
source["id"] = [8]
source["node"] = [1]
source["status"] = [1]
source["u_ref"] = [1.0]

# Construct the input data
input_data = {
    ComponentType.node: node,
    ComponentType.line: line,
    ComponentType.sym_load: sym_load,
    ComponentType.source: source
}

# Validate the input data
assert_valid_input_data(input_data)

# Create a power grid model
model = PowerGridModel(input_data)

# Run a (Newton Raphson) power flow calculation
pf_output_data = model.calculate_power_flow(
    symmetric=True, 
    error_tolerance=1e-8, 
    max_iterations=20, 
    calculation_method=CalculationMethod.newton_raphson
)

### View example data

In [ ]:
# Display the results
for component, data in pf_output_data.items():
    display(Markdown(f"### {component.title()}s (power flow):"))
    display(pd.DataFrame(data))

# Assignment 1: Number of sensors

In order to perform a state estimation some voltage and power sensors need to be added to the model.
For the calculation to be successful a minumum number of sensors is required.

- What is the minimum number of sensors to perform a state estimation on the given network?
- How many of those should be voltage sensors?


In [ ]:
# Fill in the minimal required number of sensors below
n_sensors = ...
n_voltage_sensors = ...


# Assignment 2: Collecting voltage sensor data

In this assignment print the output of the assets in the the loadflow calculation, which is needed as input for the voltage sensors. 

Hint: convert to a pandas DataFrame before printing for a better overview

In [ ]:
# Print the assets in the output data that we need for the voltage sensors
display(pd.DataFrame(pf_output_data[ComponentType.node]))

# Assignment 3: Initialize the sensors

In this assignment we will perform a state estimation based on three voltage sensors that only measure the voltage. 
If you look closely to the data, you'll notice that the number of measurements (3) is not larger than or equal to the number of unknowns (6). 
So the system is not *fully observable* and you might expect the state estimation to fail. 
However, the linear state estimation algorithm will assume the voltage angles (3) to be zero if no value is given. 
In other words, the mathematical core will give us a faulty result, without any warning! 
To prevent this, we need an observability check, which is complex, but will be added to the validation functions in the future.

- initialize the voltage sensors
- extend the input data set, with the voltage sensors
- construct a new model with the new input data
- run the state estimation calculation

In [ ]:
# TODO: Initialize 3 symmetric voltage sensors, each connected to a different node
sym_voltage_sensor = initialize_array(..., ..., ...)
sym_voltage_sensor["id"] = ...
sym_voltage_sensor["measured_object"] = ...
sym_voltage_sensor["u_sigma"] = 10.0
sym_voltage_sensor["u_measured"] = ...

# TODO: Add the sensors to the input data
input_data[...] = sym_voltage_sensor

# TODO: Validate the input data
assert_valid_input_data(..., calculation_type=..., symmetric=...)

# Create a power grid model
model = PowerGridModel(input_data)

# Run the (iterative linear) state estimation
se_output_data = model.calculate_state_estimation(
    symmetric=True, 
    error_tolerance=1e-8, 
    max_iterations=20, 
    calculation_method=CalculationMethod.iterative_linear)

# Assignment 4: Compare the results between the loadflow and state estimation

For all nodes:
- print the difference in `u` between `se_output_data` and `pf_output_data`

For all lines:
- print the difference in `p_from` between `se_output_data` and `pf_output_data`
- print the difference in `p_to` between `se_output_data` and `pf_output_data`
- print the difference in `q_from` between `se_output_data` and `pf_output_data`
- print the difference in `q_to` between `se_output_data` and `pf_output_data`

You should see that both the voltages and the *p* and *q* match quite precisely.

In [ ]:
# TODO: Print the delta u for all nodes (pf_output_data - se_output_data)
print("-------------- nodes --------------")
print("delta_u:", ...)

# TODO: Print the delta p and q for all lines (pf_output_data - se_output_data)
print("-------------- lines --------------")
print("delta_p_from:", ...)
print("delta_p_to:", ...)
print("delta_q_from:", ...)
print("delta_q_to:", ...)

# Assignment 5: Add power sensors to the model

In common power grids most voltage sensors only measure the voltage magnitude; not the angle. In this assigment we will again use the `input_data` of assignment 3 and we will connect power sensors to the model.

In our network it would be possible to connect power sensors to the lines, the loads and the source. To assign realistic measurement values to the power sensors we can use the powerflow output.

- Print the powerflow output of the lines, loads and source
- Initialize as many `sym_power_sensors` as you like (think about which data you use for which type of power sensor)
- Create a new input data set, including both voltage and power sensors
- Use the print statements of assignment 4 to compare the results

You should see that again the voltages match quite precisely (in the order of microvolts), the *p* and *q* do too (in the order of watts / VARs).

In [ ]:
# Print the lines, loads and sources
print("Lines:")
display(...)
print("Sources:")
display(...)
print("Loads:")
display(...)

In [ ]:
# TODO: Initialize as many power sensors as you like.
# Note that the sensors must added to the `input_data`, not `update_data`, as they don't exist in the model yet.sym_power_sensor = initialize_array(..., ..., ...)
sym_power_sensor = initialize_array(..., ..., ...)
sym_power_sensor["id"] = ...
sym_power_sensor["measured_object"] = ...
sym_power_sensor["measured_terminal_type"] = ...
sym_power_sensor["power_sigma"] = ...
sym_power_sensor["p_measured"] = ...
sym_power_sensor["q_measured"] = ...

# TODO: Add the sensors to the input data
input_data[...] = sym_power_sensor
# TODO: Validate the input data
assert_valid_input_data(..., calculation_type=..., symmetric=...)

# TODO: Create a new power grid model
model = PowerGridModel(...)

# Run the (iterative linear) state estimation
se_output_data_power = model.calculate_state_estimation(
    symmetric=True, 
    error_tolerance=1e-8, 
    max_iterations=20, 
    calculation_method=CalculationMethod.iterative_linear)

# TODO: Print the delta u for all nodes (se_output_data_u_angle - pf_output_data)
print("-------------- nodes --------------")
print("delta_u", ...

# TODO: Print the delta p and q for all lines (se_output_data_u_angle - pf_output_data)
print("-------------- lines --------------")
print("delta_p_from:", ...)
print("delta_p_to:", ...)
print("delta_q_from:", ...)
print("delta_q_to:", ...)

It is interesting to analyze the calculated `u_angle` as well. One thing to notice is that angles should be interpreted relatively.
A common way to do this, is to set the voltage angle of the first node to 0.0 radians and shift the rest accordingly.

In [ ]:
# Copy the angles from the powerflow output and the last state estimation output
pf_u_angles = pf_output_data[ComponentType.node]["u_angle"].copy()
se_u_angles = se_output_data_power[ComponentType.node]["u_angle"].copy()

# Print the angles
print("\nu_angle")
print("pf:", pf_u_angles)
print("se:", se_u_angles)

# Align the angles
pf_u_angles = pf_u_angles - pf_u_angles[0]
se_u_angles = se_u_angles - se_u_angles[0]

# Print the angles again
print("\nu_angle'")
print("pf:", pf_u_angles)
print("se:", se_u_angles)

# Print the deltas
print("\ndelta_u_angle")
print(se_u_angles - pf_u_angles)

# Assignment 6: Time Series Batch Calculation

Sometimes, it is desirable to see what the state of the power grid was for a number of measurements at different points in time. A typical use case is to see if the voltage or power requirements were not met over the past day.

## Voltage measurements

Let's say, we have voltage sensors with a much better temporal resolution than our power sensors. To simulate such a situation, we generate some random voltage measurements based on the input data used before, but re-use the power sensor readings.

In [ ]:
n_scenarios = 96
n_sensors = len(input_data[ComponentType.sym_voltage_sensor])
sensor_id = input_data[ComponentType.sym_voltage_sensor]["id"]
sensor_u_measured = sym_voltage_sensor["u_measured"]
measurements = np.tile(sensor_u_measured, (n_scenarios, 1)) + np.random.normal(scale=100, size=(n_scenarios, n_sensors))
dti = pd.date_range("2022-01-01", periods=n_scenarios, freq="15min")
df_voltage_measurements = pd.DataFrame(measurements, columns=sensor_id, index=dti)
display(df_voltage_measurements)

## Run Time Series Calculation

We want to run a time-series state estimation using the dataframe.

* Convert the measurements to the compatible batch update dataset.
* Run the batch calculation

In [ ]:
# TODO: Initialize empty measurements
sym_voltage_measurements = initialize_array(..., ..., ...)

# TODO: Set the attributes for the batch calculation
# (assume u_sigma and u_angle_measurement are as before)
sym_voltage_measurements["id"] = ...
sym_voltage_measurements["u_measured"] = ...

update_data = {ComponentType.sym_voltage_sensor: sym_voltage_measurements}

In [ ]:
# Validating batch data can take a long time.
# It is recommended to only validate batch data when you run into trouble.
assert_valid_batch_data(
    input_data=input_data,
    update_data=update_data,
    calculation_type=CalculationType.state_estimation,
)

In [ ]:
output_data = model.calculate_state_estimation(...)

### Extracting load information from batch results

We are trying to determine whether any user had a significant fluctuation in load requirements over the course of this day.

* Determine the minimal and maximal power load and their ratio.
* Was the fluctuation in power requirements large during this simulated day?

In [ ]:
# TODO: Extract the power field
power_load = output_data[ComponentType.sym_load][...]
# TODO: Calculate the max and min for each user
max_p = power_load.max(...)
min_p =  power_load.min(...)
print("max power load:", max_p)
print("min power load:", min_p)
print("ratio:", max_p / min_p)

### Plotting batch results

Lets say we wish to plot the loading of the `line with id 4` vs time. We can use matplotlib to do so.

**Note:** The grid and results are randomly generated so dont be alarmed to see loading >100% or any other unrealistic results.

In [ ]:
from matplotlib import pyplot as plt

# TODO: Prepare data to be plotted. We wish to plot the loading of line with id 4 vs time.
line_4_idx = np.where(... == 4)
result_loading = output_data[ComponentType.line]["loading"][...]

plt.plot(result_loading)
plt.title('Loading of line with id 2007')
plt.xlabel('Time')
plt.ylabel('Loading')
plt.show()